# AST4930 Homework Assignment #7 (20 pts + 2 pts)

### Instruction:

In this homework, we will use the SDSS DR16 data (https://www.sdss.org/dr16/) and build deep neural networks which will be used to classify objects into galaxies, quasars, and stars. This is the same dataset we used for Homework Assignment #2 and #3. 

Below you will find instruction to this homework assignment.

1. I learned that when I download your .ipynb file from Canvas, it automatically adds your name to the file name. So you don't have to put your last name manually as you did last time.

2. As you will see, the Notebook is partially filled. Fill in the empty cells. 

3. Note that there are questions you will have to answer. They will start with "Q: ..." The answers will be graded so make sure you answer all the questions.

4. It's fine you work on somewhere else than HiPerGator. But when I grade your homework, I will run your Jupyter Notebook on HiPerGator. So before you submit your homework, make sure you test it on HiPerGator and see if you get any error messages. For this particular homework assignment, we are concerned about the computing time so I'd like you all to use the HiPerGator resources for a fair comparison. 

5. It's fine if you use someone else's code you found on Internet, but make sure you write down the source. You don't have to cite me when you use anything from AST4930 Notebook. But as we learned from Homework Assignment #2, I do make mistakes. So please double-check before you copy and paste my script!

6. It's a common sense but just to make sure... when you make a figure, make sure they are readable/understandable. If you cannot see the title, labels, data points, or if you don't understand the point of the figure, I won't probably be able to see/understand them either!

6. Submit your final .ipynb file via Canvas before the due date.

### Let's load the data. This is the same dataset we used for Homework Assignment #2 and #3.

In [ ]:
import pandas as pd 

data = pd.read_csv('/blue/ast4930/share/SDSS.csv')

### Let's have a look at the data to gain insights.

In [2]:
data

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1237655107828973746,138.937318,49.779249,17.32152,16.28594,15.88471,15.66384,15.46604,2243,301,3,152,862475375345690624,GALAXY,0.028514,766,52247,131
1,1237678584114511952,31.594644,22.451246,18.13019,17.47997,16.97868,16.75768,16.83714,7709,301,3,65,2303710287351539712,QSO,1.694959,2046,53327,433
2,1237680298881122360,333.991221,19.977552,18.94798,18.80473,18.75522,18.72968,18.66552,8108,301,5,79,8532228490187264000,QSO,2.046520,7578,56956,578
3,1237650761319514319,186.201096,-2.808180,19.22969,18.04638,17.37830,16.99285,16.76479,1231,301,3,113,376103378880587776,GALAXY,0.000000,334,51993,192
4,1237648722280317126,124.814240,0.651496,18.76061,17.28682,16.74055,16.55152,16.45902,756,301,5,60,2316149895234349056,STAR,0.000196,2057,53816,632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,1237673758185554973,100.889697,29.106283,19.17071,18.21106,17.80099,17.63203,17.53079,6585,301,6,121,3013004977846118400,STAR,0.000160,2676,54179,352
149996,1237671262268752282,124.964450,9.635759,19.38253,17.49415,16.55436,16.13420,15.80408,6004,301,5,61,2728125362876737536,GALAXY,0.110984,2423,54149,254
149997,1237673756573762584,98.821075,26.946457,18.90948,17.73591,17.31252,17.16612,17.07103,6585,301,3,103,3035579600630474752,STAR,0.000097,2696,54167,558
149998,1237678877782966354,320.766012,9.993010,17.63177,16.10855,15.35884,15.21912,15.11279,7777,301,6,66,821986413082863616,STAR,-0.000218,730,52466,289


### In addition to ugriz magnitudes, we will use colors (u-g, g-r, r-i, i-z) so let's make those arrays.

In [3]:
data['u-g'] = data.u-data.g
data['g-r'] = data.g-data.r
data['r-i'] = data.r-data.i
data['i-z'] = data.i-data.z

### Features and target

In [4]:
X = data[['u','g','r','i','z','u-g','g-r','r-i','i-z','redshift']]
y = data['class']

In [5]:
X

,u,g,r,i,z,u-g,g-r,r-i,i-z,redshift
0,17.32152,16.28594,15.88471,15.66384,15.46604,1.03558,0.40123,0.22087,0.19780,0.028514
1,18.13019,17.47997,16.97868,16.75768,16.83714,0.65022,0.50129,0.22100,-0.07946,1.694959
2,18.94798,18.80473,18.75522,18.72968,18.66552,0.14325,0.04951,0.02554,0.06416,2.046520
3,19.22969,18.04638,17.37830,16.99285,16.76479,1.18331,0.66808,0.38545,0.22806,0.000000
4,18.76061,17.28682,16.74055,16.55152,16.45902,1.47379,0.54627,0.18903,0.09250,0.000196
...,...,...,...,...,...,...,...,...,...,...
149995,19.17071,18.21106,17.80099,17.63203,17.53079,0.95965,0.41007,0.16896,0.10124,0.000160
149996,19.38253,17.49415,16.55436,16.13420,15.80408,1.88838,0.93979,0.42016,0.33012,0.110984
149997,18.90948,17.73591,17.31252,17.16612,17.07103,1.17357,0.42339,0.14640,0.09509,0.000097
149998,17.63177,16.10855,15.35884,15.21912,15.11279,1.52322,0.74971,0.13972,0.10633,-0.000218


### Below is to convert the labels to "one-hot encoded" integers (see e.g., https://en.wikipedia.org/wiki/One-hot)

In [6]:
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras as keras

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

# convert integers to dummy variables (i.e. one-hot encoded)
dummy_y = keras.utils.to_categorical(encoded_y)

In [7]:
y

0         GALAXY
1            QSO
2            QSO
3         GALAXY
4           STAR
           ...  
149995      STAR
149996    GALAXY
149997      STAR
149998      STAR
149999    GALAXY
Name: class, Length: 150000, dtype: object

In [8]:
dummy_y

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

### Now, build DNNs, optimize hyperparameters, train and evaluate DNNs.

This is the last homework of the course, so I am not going to guide you step by step like I did before. You should be able to do this by now :) Things you should include at the very least are: hyperparameter optimization, loss/accuracy vs. epoch plots, confusion matrix.


### Q: Before you get started, do we need feature scaling for DNNs or not? Why?

### Evaluate your final model. If you get >= 99% accuracy on your test dataset, you get 2 extra points.

### Q: How does the model performance compare with what you got in homework #2 and #3? What pros and cons of DNNs did you find? 